# Simulating spin-1/2 chains on current quantum computers

In [ ]:
import numpy as np
import scipy as sp
from functools import partial
from random import uniform

#Qiskit libraries
from qiskit import QuantumCircuit,QuantumRegister, transpile, Aer, IBMQ,execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator
import qiskit.test.mock

#matplotlib for visualization
import matplotlib.pyplot as plt 

# 1. Quantum Circuit
## 1.1 Initial state
IBM quantum computers and their simulators start each execution in the state |00...0>. The circuits in this section create the following initial states
- neel initial state: |...🠕🠗🠕🠗🠕...>
- domain wall initial state: |...🠗🠗🠗🠕🠕🠕...>

([p.2](https://www.nature.com/articles/s41534-019-0217-0.pdf#page=2))

In [ ]:
def neel_initial_state(num_qubits):
    """prepare the neel initial state

    Parameters:
    num_qubits: total number of qubits

    Returns:
    Circuit preparing the neel initial state from the IBM initial State 
    """
    
    qreg = QuantumRegister(num_qubits,'q')
    qc = QuantumCircuit(qreg, name='neill initialization')
    for i in range(num_qubits//2):
        qc.x(qreg[2*i + 1])
    return qc

def domainwall_initial_state(num_qubits):
    """prepare the domain wall initial state

    Parameters:
    num_qubits: total number of qubits

    Returns:
    Circuit preparing the domain wall initial state from the IBM initial State 
    """

    qreg = QuantumRegister(num_qubits,'q')
    qc = QuantumCircuit(qreg, name='domain wall initialization')
    for i in range((-(-num_qubits//2))):
        qc.x(qreg[i])
    return qc

## 1.2 Single time step
circuit for a single time step, implemented with gates supported by IBM hardware
1. decomposition of the time propagator into 1- and 2-qubit gates using trotterization ([pp.7-8](https://www.nature.com/articles/s41534-019-0217-0.pdf#page=7), [Fig. 1](https://www.nature.com/articles/s41534-019-0217-0.pdf#page=2), Eq 15, Eq 16)
2. decomposition of 2-qubit gates into gates available on IBM hardware (single qubit gates and CNOT) ([p.9](https://www.nature.com/articles/s41534-019-0217-0.pdf#page=9), Eq 28-32)

In [ ]:
def N_gate(α,β,γ):
    """ N-Gate     (see Eq 28,29,30)

    Parameters:
    α: factor for Pauli-X
    β: factor for Pauli-Y
    γ: factor for Pauli-Z

    Returns:
    Circuit implementing exp(i(αX + βY + γZ)) on two qubits    
    """

    qreg = QuantumRegister(2,'q')
    qc = QuantumCircuit(qreg, name='N')
    
    #N gate
    qc.rz(-np.pi/2, qreg[1])
    qc.cx(qreg[1],qreg[0])
    qc.rz(np.pi/2 - 2*γ ,qreg[0])
    qc.ry(2*α - np.pi/2, qreg[1])
    qc.cx(qreg[0],qreg[1])
    qc.ry(np.pi/2 - 2*β, qreg[1])
    qc.cx(qreg[1],qreg[0])
    qc.rz(np.pi/2,qreg[0])
            
    return qc

def MN_gate(α,β):
    """ Magic N-Gate (N-Gate optimized for only 2 parameters, see Eq 31,32)

    Parameters:
    α: factor for Pauli-X
    β: factor for Pauli-Y

    Returns:
    Circuit implementing exp(i(αX + βY)) on two qubits
    """

    qreg = QuantumRegister(2,'q')
    qc = QuantumCircuit(qreg, name='MN')
    
    #basis change Y -> Z
    qc.h(qreg[0])
    qc.h(qreg[1])
    qc.sdg(qreg[0])
    qc.sdg(qreg[1])
    qc.h(qreg[0])
    qc.h(qreg[1])
    
    #simpler N gate
    qc.cx(qreg[0],qreg[1])
    qc.rx(-2*α, qreg[0])
    qc.rz(-2*β, qreg[1])
    qc.cx(qreg[0],qreg[1])
    
    #basis change Z -> Y
    qc.h(qreg[0])
    qc.h(qreg[1])
    qc.s(qreg[0])
    qc.s(qreg[1])
    qc.h(qreg[0])
    qc.h(qreg[1])
    
    return qc
 
def time_step_simple(num_qubits,delta_t,J,U,h):
    """ Basic Trotter decomposition with order 2 of the Time evolution operator 
    for one time step of length delta_t and the Hamiltonian specified by parameters J,U,h (see Eq 15)

    Parameters:
    num_qubits: total number of qubits
    delta_t: length of each time step
    J,U,h: parameters specifying the Hamiltonian (Eq 3)

    Returns:
    Circuit implementing the time evolution operator for one time step
    """

    qreg = QuantumRegister(num_qubits,'q')
    qc = QuantumCircuit(qreg, name='time step')
        
    #interacting gates
    #simplified circuit if U is 0 using the Magic N gate
    if J != 0 and U == 0:
        #odd indices
        for j in range((num_qubits-1)//2):
            qc.append(MN_gate(delta_t*J, delta_t*J),
                      [qreg[2*j + 1], qreg[2*j + 2]])
        #even indices
        for j in range(num_qubits//2):
            qc.append(MN_gate(delta_t*J,delta_t*J),[qreg[2*j],qreg[2*j + 1]])
        
    #regular circuit using the N gate
    elif J != 0:
        #odd indices
        for j in range((num_qubits-1)//2):
            qc.append(N_gate(delta_t*J, delta_t*J, -delta_t*U),
                      [qreg[2*j + 1], qreg[2*j + 2]])
        #even indices
        for j in range(num_qubits//2):
            qc.append(N_gate(delta_t*J,delta_t*J,-delta_t*U),[qreg[2*j],qreg[2*j + 1]])

    #single qubit gates
    for j,hj in enumerate(h):
        qc.rz(2*delta_t*hj, qreg[j])
    return qc

def time_step_symmetric(num_qubits,delta_t,J,U,h):
    """ Symmetric Trotter decomposition (order 3) of the Time evolution operator 
    for one time step of length delta_t and the Hamiltonian specified by parameters J,U,h
    (see Eq 16)

    Parameters:
    num_qubits: total number of qubits
    delta_t: length of each time step
    J,U,h: parameters specifying the Hamiltonian (Eq 3)

    Returns:
    Circuit implementing the time evolution operator for one time step
    """

    qreg = QuantumRegister(num_qubits,'q')
    qc = QuantumCircuit(qreg, name='time step')
    
    #single qubit gates
    for j,hj in enumerate(h):
        qc.rz(delta_t*hj, qreg[j])
        
    #interacting gates
    #simplified circuit if U is 0 using only 2 CNOT per step
    if J != 0 and U == 0:
        #even indices
        for j in range(num_qubits//2):
                qc.append(MN_gate(delta_t/2*J,delta_t/2*J),[qreg[2*j],qreg[2*j + 1]])
        #odd indices
        for j in range((num_qubits-1)//2):
               qc.append(MN_gate(delta_t*J,delta_t*J),[qreg[2*j + 1],qreg[2*j + 2]])
        #even indices
        for j in range(num_qubits//2):
                qc.append(MN_gate(delta_t/2*J,delta_t/2*J),[qreg[2*j],qreg[2*j + 1]])
        
    #regular circuit using 3 CNOT per step
    elif J != 0:
        #even indices
        for j in range(num_qubits//2):
                qc.append(N_gate(delta_t/2*J,delta_t/2*J,-delta_t/2*U),[qreg[2*j],qreg[2*j + 1]])
        #odd indices
        for j in range((num_qubits-1)//2):
               qc.append(N_gate(delta_t*J,delta_t*J,-delta_t*U),[qreg[2*j + 1],qreg[2*j + 2]])
        #even indices
        for j in range(num_qubits//2):
                qc.append(N_gate(delta_t/2*J,delta_t/2*J,-delta_t/2*U),[qreg[2*j],qreg[2*j + 1]])

    #single qubit gates
    for j,hj in enumerate(h):
        qc.rz(delta_t*hj, qreg[j])
    return qc

## 1.3 Multi timestep circuit
combined circuit repeating time steps until the desired total simulation time is reached ([p.2](https://www.nature.com/articles/s41534-019-0217-0.pdf#page=2), Fig. 1a)

In [ ]:
def compose_circuit(num_qubits,symmetric, delta_t, time_steps,J,U,h, initial_state,rest_time=0):
    """ Compose Circuit for a fixed time by combining single time step circuits
    for the total time t = delta_t * time_steps + rest_time

    Parameters:
    num_qubits: total number of qubits
    symmetric: if true do a symmetric trotter decomposition for the single time steps, else basic
    delta_t: length of each trotter step
    time_steps: number of trotter steps for the circuit
    rest_time: time to add for the final trotter step (Eq 10)
    initial_state: circuit preparing the initial state (neel or domain wall)
    J,U,h: parameters specifying the Hamiltonian (Eq 3)

    Returns:
    Circuit implementing the time evolution operator for the total time 
    """
    
    qreg = QuantumRegister(num_qubits,'q')
    qc = QuantumCircuit(qreg, name='simulation')
    
    qc.append(initial_state,qreg)
    if not symmetric:
        for i in range(time_steps):
            qc.append(time_step_simple(num_qubits,delta_t,J,U,h),qreg)
        qc.append(time_step_simple(num_qubits,rest_time,J,U,h),qreg)
    else:
        for i in range(time_steps):
            qc.append(time_step_symmetric(num_qubits,delta_t,J,U,h),qreg)
        qc.append(time_step_symmetric(num_qubits,rest_time,J,U,h),qreg)
    qc.measure_all()
    return qc


# 2. Run Simulations 
simulations of the physical system using different backends and numerical solutions for reference

note: `numerical_reduced_trotter`, `simulate_real` and `run_on_hardware` use a reduced number of trotter steps with additional shorter time steps in between to obtain more data points with a shallower circuit ([p.7](https://www.nature.com/articles/s41534-019-0217-0.pdf#page=7), [Eq 10](https://www.nature.com/articles/s41534-019-0217-0.pdf#page=7))

In [ ]:
#Helper methods for the numerical solutions
def ox(j, N):
    '''Matrix representation of X Gate acting on qubit j'''
    return np.kron(np.kron(np.identity(2**j, dtype='complex128'), np.array([[0, 1], [1, 0]])), np.identity(2**(N-j-1), dtype='complex128'))
def oy(j, N):
    '''Matrix representation of Y Gate acting on qubit j'''
    return np.kron(np.kron(np.identity(2**j, dtype='complex128'), np.array([[0, 0-1j], [0+1j, 0]])), np.identity(2**(N-j-1), dtype='complex128'))
def oz(j, N):
    '''Matrix representation of Z Gate acting on qubit j'''
    return np.kron(np.kron(np.identity(2**j, dtype='complex128'), np.array([[1, 0], [0, -1]])), np.identity(2**(N-j-1), dtype='complex128'))


def init(initial_state, num_qubits):
    '''vector representation of an initial state (domain wall or neel)'''

    state = np.array([0]*(2**num_qubits))
    match initial_state:
        #domain wall initial state
        case 'domain wall':
                state[int(2**-(-num_qubits//2) - 1)] = 1
        #neel initial state
        case _: state[int('01'*(num_qubits//2) + ('0' if num_qubits % 2 != 0 else ''), base=2)] = 1
    return state

def trotter_step(t,num_qubits,symmetric,J,U,h):
    '''matrix representation of one trotter step'''

    #regular trotter step (order 2)
    if(not symmetric):
        _A = np.array([[0]*(2**num_qubits)] *
                        (2**num_qubits), dtype='complex128')
        for j in range(num_qubits):
            _A += h[j]*oz(j, num_qubits)
        A = sp.linalg.expm(-(0+1j)*t*_A)

        _B_even = np.array([[0]*(2**num_qubits)] *
                            (2**num_qubits), dtype='complex128')
        for j in range(0, num_qubits - 1, 2):
            _B_even += -J*(ox(j, num_qubits).dot(ox(j+1, num_qubits)) + oy(j, num_qubits).dot(
                oy(j+1, num_qubits))) + U*oz(j, num_qubits).dot(oz(j+1, num_qubits))
        B_even = sp.linalg.expm(-(0+1j)*t*_B_even)

        _B_odd = np.array([[0]*(2**num_qubits)] *
                            (2**num_qubits), dtype='complex128')
        for j in range(1, num_qubits - 1, 2):
            _B_odd += -J*(ox(j, num_qubits).dot(ox(j+1, num_qubits)) + oy(j, num_qubits).dot(
                oy(j+1, num_qubits))) + U*oz(j, num_qubits).dot(oz(j+1, num_qubits))
        B_odd = sp.linalg.expm(-(0+1j)*t*_B_odd)
        return A.dot(B_even.dot(B_odd))

    #symmetric trotter step (order 3)
    else:
        _A = np.array([[0]*(2**num_qubits)] *
                        (2**num_qubits), dtype='complex128')
        for j in range(num_qubits):
            _A += h[j]*oz(j, num_qubits)
        A = sp.linalg.expm(-(0+1j)*t/2*_A)

        _B_even = np.array([[0]*(2**num_qubits)] *
                            (2**num_qubits), dtype='complex128')
        for j in range(0, num_qubits - 1, 2):
            _B_even += -J*(ox(j, num_qubits).dot(ox(j+1, num_qubits)) + oy(j, num_qubits).dot(
                oy(j+1, num_qubits))) + U*oz(j, num_qubits).dot(oz(j+1, num_qubits))
        B_even = sp.linalg.expm(-(0+1j)*t/2*_B_even)

        _B_odd = np.array([[0]*(2**num_qubits)] *
                            (2**num_qubits), dtype='complex128')
        for j in range(1, num_qubits - 1, 2):
            _B_odd += -J*(ox(j, num_qubits).dot(ox(j+1, num_qubits)) + oy(j, num_qubits).dot(
                oy(j+1, num_qubits))) + U*oz(j, num_qubits).dot(oz(j+1, num_qubits))
        B_odd = sp.linalg.expm(-(0+1j)*t*_B_odd)
        return A.dot(B_even.dot(B_odd.dot(B_even.dot(A))))


def numerical(num_qubits, time, time_steps, J, U, h, initial_state):
    """ Simulate time evolutuion of the system numerically using the time propagator

    Parameters:
    num_qubits: total number of qubits
    delta_t: length of each trotter step
    time_steps: number of trotter steps for the circuit
    initial_state: either 'domain wall' or 'neel'
    J,U,h: parameters specifying the Hamiltonian (Eq 3)

    Returns:
    numerical result for the time evolution of the physical system defined by J,U,h
    (list with entries {t:<time>, backend:<backend_name>, data:<end state>} for each time step
    with <end state> having the format {<basis state>:<probability>})
    """

    delta_t = time/time_steps
    results = []

    def time_propagator(t):
        H = np.array([[0]*(2**num_qubits)]*(2**num_qubits), dtype='complex128')
        for j in range(num_qubits - 1):
            H += -J*(ox(j, num_qubits).dot(ox(j+1, num_qubits)) + oy(j, num_qubits).dot(
                oy(j+1, num_qubits))) + U*oz(j, num_qubits).dot(oz(j+1, num_qubits))
            H += h[j]*oz(j, num_qubits)
        H += h[num_qubits - 1]*oz(num_qubits-1, num_qubits)
        return sp.linalg.expm(-(0+1j)*t*H)

    #calculate time propagator matrix
    state = init(initial_state, num_qubits)
    
    for i in range(time_steps + 1):
        res = time_propagator(delta_t*i).dot(state.copy())
        results.append({'t': delta_t*i, 'backend':'numerical', 'data': {format(j,'0' + str(num_qubits) + 'b') : np.abs(res[j])**2 for j in range(np.size(res)) if np.abs(res[j] != 0)}})
    return results


def numerical_trotter(num_qubits,symmetric, time, time_steps, J, U, h, initial_state):
    """ Simulate time evolutuion of the system numerically using the trotterized time propagator

    Parameters:
    num_qubits: total number of qubits
    symmetric: if true do a symmetric trotter decomposition (order 3), else the basic one (order 2)
    delta_t: length of each trotter step
    time_steps: number of trotter steps for the circuit
    initial_state: either 'domain wall' or 'neel'
    J,U,h: parameters specifying the Hamiltonian (Eq 3)

    Returns:
    numerical result for the trotterized time evolution of the physical system defined by J,U,h
    (list with entries {t:<time>, backend:<backend_name>, data:<end state>} for each time step
    with <end state> having the format {<basis state>:<probability>})
    """

    delta_t = time/time_steps
    results = []

    state = init(initial_state, num_qubits)

    for i in range(time_steps + 1):
            res = np.linalg.matrix_power(trotter_step(delta_t,num_qubits,symmetric,J,U,h), i).dot(state.copy())
            results.append({'t': delta_t*i, 'backend':'numerical', 'data': {format(j,'0' + str(num_qubits) + 'b') : np.abs(res[j])**2 for j in range(np.size(res)) if np.abs(res[j] != 0)}})
    return results

def numerical_reduced_trotter(num_qubits,symmetric, time, time_steps, J, U, h, initial_state):
    """ Simulate time evolutuion of the system numerically using the trotterized time propagator and fewer trotter steps
    between every two trotter steps, 4 data points are added by varying the length of the final trotter step (Eq 10)

    Parameters:
    num_qubits: total number of qubits
    symmetric: if true do a symmetric trotter decomposition (order 3), else the basic one (order 2)
    delta_t: length of each trotter step
    time_steps: number of trotter steps for the circuit
    initial_state: either 'domain wall' or 'neel'
    J,U,h: parameters specifying the Hamiltonian (Eq 3)

    Returns:
    numerical result for the trotterized time evolution of the physical system defined by J,U,h
    (list with entries {t:<time>, backend:<backend_name>, data:<end state>} for each time step
    with <end state> having the format {<basis state>:<probability>})
    """

    results = []
    trotter_steps = time_steps//4
    state = init(initial_state,num_qubits)

    delta_t = time/trotter_steps
    time_between_datapoints = time/time_steps
    datapoints_per_trotterstep = time_steps//trotter_steps
    for i in range(trotter_steps):
        for j in range(datapoints_per_trotterstep):
            num_trottersteps = i
            rest_time = j*time_between_datapoints
            res = trotter_step(rest_time,num_qubits,symmetric,J,U,h).dot(np.linalg.matrix_power(trotter_step(delta_t,num_qubits,symmetric,J,U,h), i)).dot(state.copy())
            results.append({'t': delta_t*i + rest_time, 'backend':'numerical trotter', 'data': {format(j,'0' + str(num_qubits) + 'b') : np.abs(res[j])**2 for j in range(np.size(res)) if np.abs(res[j] != 0)}})
    res = (np.linalg.matrix_power(trotter_step(delta_t, num_qubits, symmetric, J, U, h), trotter_steps)).dot(state.copy())
    results.append({'t': time, 'backend':'numerical trotter', 'data': {format(j,'0' + str(num_qubits) + 'b') : np.abs(res[j])**2 for j in range(np.size(res)) if np.abs(res[j] != 0)}})
    return results

def simulate_ideal_trotter(num_qubits,symmetric, time, time_steps,J,U,h,initial_state):
    """Statevector simulation with regular Trotterization
    Simulate the circuit without quantum errors and one trotter step between every two datapoints
    backend: aer statevector simulator

    Parameters:
    num_qubits: total number of qubits
    symmetric: if true do a symmetric trotter decomposition (order 3), else the basic one (order 2)
    delta_t: length of each trotter step
    time_steps: number of trotter steps for the circuit
    initial_state: circuit preparing the initial state (neel or domain wall)
    J,U,h: parameters specifying the Hamiltonian (Eq 3)

    Returns:
    data obtained in the simulation 
    (list with entries {t:<time>, backend:<backend_name>, data:<counts>} for each time step
    with <counts> having the format {<basis state>:<number of measurement results>})
    """

    delta_t = time/time_steps
    simulator = Aer.get_backend('statevector_simulator')
    results = []
    for i in range(time_steps + 1):
        qc = transpile(compose_circuit(num_qubits,symmetric, delta_t, i,J,U,h,initial_state), simulator)
        step_result = simulator.run(qc, shots=8192).result()
        results.append({'t': delta_t*i, 'backend': 'statevector_simulator',
                       'data': step_result.get_counts(qc)})
    return results

def simulate_ideal(num_qubits,symmetric, time, time_steps, trotter_steps,J,U,h,initial_state):
    """
    Error-free simulation with reduced Trotterization
    Simulate the circuit without quantum errors and additional datapoints between trotter steps (Eq.10)
    backend: aer simulator

    Parameters:
    num_qubits: total number of qubits
    symmetric: if true do a symmetric trotter decomposition (order 3), else the basic one (order 2)
    delta_t: length of each trotter step
    time_steps: number of trotter steps for the circuit
    initial_state: circuit preparing the initial state (neel or domain wall)
    J,U,h: parameters specifying the Hamiltonian (Eq 3)

    Returns:
    data obtained in the simulation 
    list with entries {t:<time>, backend:<backend_name>, data:<counts>} for each time step
    """

    trotter_steps = time_steps//4

    delta_t = time/trotter_steps
    time_between_datapoints = time/time_steps
    datapoints_per_trotterstep = time_steps//trotter_steps
    simulator = Aer.get_backend('aer_simulator')
    results = []

    for i in range(trotter_steps):
        for j in range(datapoints_per_trotterstep):
            num_trottersteps = i
            rest_time = j*time_between_datapoints
            qc = transpile(compose_circuit(num_qubits,symmetric, delta_t, num_trottersteps,J,U,h,initial_state,rest_time), simulator)
            step_result = simulator.run(qc, shots=8192).result()
            results.append({'t': delta_t*i + time_between_datapoints*j, 'backend':'aer_simulator', 'data':step_result.get_counts(qc)})
    qc = transpile(compose_circuit(num_qubits,symmetric, delta_t, trotter_steps,J,U,h,initial_state,0), simulator)
    step_result = simulator.run(qc, shots=8192).result()
    results.append({'t': time, 'backend':'aer_simulator', 'data':step_result.get_counts(qc)})
    return results

def simulate_real(num_qubits,symmetric, time, time_steps, trotter_steps,J,U,h,initial_state,device):
    """ Simulation mimicking the errors of a real device
    Simulate real device with additional datapoints between trotter steps (Eq.10)
    backend: qiskit fakeProvider

    Parameters:
    num_qubits: total number of qubits
    symmetric: if true do a symmetric trotter decomposition (order 3), else the basic one (order 2)
    delta_t: length of each trotter step
    time_steps: number of trotter steps for the circuit
    initial_state: circuit preparing the initial state (neel or domain wall)
    device: name of the device to simulate
    J,U,h: parameters specifying the Hamiltonian (Eq 3)

    Returns:
    data obtained in the simulation 
    list with entries {t:<time>, backend:<backend_name>, data:<counts>} for each time step
    """

    trotter_steps = time_steps//4

    delta_t = time/trotter_steps
    time_between_datapoints = time/time_steps
    datapoints_per_trotterstep = time_steps//trotter_steps
    simulator = qiskit.test.mock.FakeProvider().get_backend(device)
    results = []

    for i in range(trotter_steps):
        for j in range(datapoints_per_trotterstep):
            num_trottersteps = i
            rest_time = j*time_between_datapoints
            qc = transpile(compose_circuit(num_qubits,symmetric, delta_t, num_trottersteps,J,U,h,initial_state,rest_time), simulator)
            step_result = simulator.run(qc, shots=8192).result()
            results.append({'t': delta_t*i + time_between_datapoints*j,
                        'backend': device, 'data': step_result.get_counts(qc)})
    qc = transpile(compose_circuit(num_qubits,symmetric, delta_t, trotter_steps,J,U,h,initial_state,0), simulator)
    step_result = simulator.run(qc, shots=8192).result()
    results.append({'t': time,'backend': device, 'data': step_result.get_counts(qc)})
    return results


def run_on_hardware(num_qubits, symmetric, time, time_steps, trotter_steps, J, U, h, initial_state,device,provider):
    """ Run the circuit on real quantum hardware with with additional datapoints between trotter steps (Eq.10)
    backend: IBM quantum computer

    Parameters:
    num_qubits: total number of qubits
    symmetric: if true do a symmetric trotter decomposition (order 3), else the basic one (order 2)
    delta_t: length of each trotter step
    time_steps: number of trotter steps for the circuit
    initial_state: circuit preparing the initial state (neel or domain wall)
    device: name of the device to run on
    J,U,h: parameters specifying the Hamiltonian (Eq 3)

    Returns:
    data obtained in the simulation 
    list with entries {t:<time>, backend:<backend_name>, data:<counts>} for each time step

    """
    trotter_steps = time_steps//4

    delta_t = time/trotter_steps
    time_between_datapoints = time/time_steps
    datapoints_per_trotterstep = time_steps//trotter_steps
    device = provider.get_backend(device)
    results = []

    for i in range(trotter_steps):
        for j in range(datapoints_per_trotterstep):
            num_trottersteps = i
            rest_time = j*time_between_datapoints        
            qc = transpile(compose_circuit(num_qubits, symmetric, delta_t,
                        num_trottersteps, J, U, h, initial_state, rest_time), device)
            job = execute(qc, backend=device, shots=8192)
            from qiskit.tools.monitor import job_monitor
            job_monitor(job)
            step_result = job.result()
            results.append({'t': delta_t*i + time_between_datapoints*j, 'backend': device,
                        'data': step_result.get_counts(qc)})
    qc = transpile(compose_circuit(num_qubits, symmetric, delta_t,
                                   trotter_steps, J, U, h, initial_state, 0), device)
    job = execute(qc, backend=device, shots=8192)
    from qiskit.tools.monitor import job_monitor
    job_monitor(job)
    step_result = job.result()
    results.append({'t': time, 'backend': device,
                    'data': step_result.get_counts(qc)})
    return results


# 3. Data processing
calculate the probability amplitudes and expectation values of different observables from the measured data ([pp. 2-3](https://www.nature.com/articles/s41534-019-0217-0.pdf#page=2), [Eq 4,5,6,7](https://www.nature.com/articles/s41534-019-0217-0.pdf#page=2))

In [ ]:
def clean_data(results):
    """ clean data by replacing absoulute frequencies of the measurement results with probabilities

    Parameters:
    results: data obtained in a simulation 
             list with entries {t:<time>, backend:<backend_name>, data:<counts>} for each time step

    Returns:
    input data with total counts normalized to probabilities
    """

    cleaned_data = []
    for result in results:
        cleaned_data.append({'t': result['t'], 'backend': result['backend'], 'data':{k: v/8192 for k, v in result['data'].items()}})
    return cleaned_data   

def clean_data_error_mitigation(results):
    """ clean data by replacing absoulute frequencies of the measurement results with probabilities
    and simple error mitigation by discarding states that violate the conservation of total spin (p. 8)

    Parameters:
        results: data obtained in a simulation 
                 list with entries {t:<time>, backend:<backend_name>, data:<counts>} for each time step

    Returns:
    input data with total counts normalized to probabilities and only containing physical states
    """

    cleaned_data = []
    for result in results:
        count_total = 0
        physical_results = {}
        #remove unphysical states
        for state,count in result['data'].items():
            if state.count('0') == int(len(state)/2):
                physical_results[state] = count
                count_total += count
        #normalize counts to probabilities
        cleaned_data.append({'t': result['t'], 'backend': result['backend'], 'data':{k: v/count_total for k, v in physical_results.items()}})
    return cleaned_data


"""
The following methods calculate the expectation values specified in Eq 5 - 7
optimized by calculating only the effect of the observables instead of assembling matrices

note: unlike in the paper, indexing of the qubits starts at 0
note: the meaning of states 0 and 1 is flipped in the calculation in order to be consistent with
the experimental data provided by A. Smith et al
"""

def σ(j,state):
    """ Expectation value for the observable Pauli Z (Eq 4)
    z-spin for one qubit 

    Parameters:
    j: index of the qubit for which the expectation value is calculated (starting at 0)
    state: one datapoint of cleaned simulation result

    Returns:
    expectation value for z-spin for qubit at site j
    """

    res = 0
    for basis_state,probability in state.items():
        res += probability * (-1 if basis_state[j] == '0' else 1)
    return res


def σ_2(j,k,state):
    """ Helper observable: Pauli Z on two qubits
    z-spin for two-qubit-system

    Parameters:
    j,k: indices of the qubits
    state: one datapoint of cleaned simulation result

    Returns:
    expectation value for the z-spin of a two qubit system at sites j and k
    """

    if (j == k): return 1
    res = 0
    for basis_state,probability in state.items():
        res += probability * (1 if basis_state[j] == basis_state[k] else -1)
    return res

def expectation_value_Nhalf(state):
    """ Expectation value for the observable N_half (Eq 5)
    measure for how much the initial state (domain wall) is disturbed

    Parameters:
    state: one datapoint of cleaned simulation result

    Returns:
    expectation value for N_half
    """

    num_qubits = len(next(iter(state)))
    res = 0
    for j in range(num_qubits//2):
        for basis_state,probability in state.items():
            res += probability *(1 if basis_state[j] =='1' else 0)
    return res

def expectation_value_C(j,k,state):
    """ Expectation value for the connected equal-time correlator (Eq 6)
    correlator between the z-spins of two distant qubits 
    (absolute value: measures only correlation strength, not direction)

    Parameters:
    j,k: indices of the qubits
    state: one datapoint of cleaned simulation result

    Returns:
    absolute value of the connected equal-time correlator 
    """

    return np.abs(σ_2(j,k,state) - σ(j,state)*σ(k,state))

def expectation_value_Qf(state):
    """ quantum Fisher information (Eq 7)
    measure for the overall entanglement in the system

    Parameters:
    state: one datapoint of cleaned simulation result

    Returns:
    quantum Fisher information normalized by the number of qubits
    """

    num_qubits = len(next(iter(state)))
    def s(site):
        return 1 if site < num_qubits/2 else -1
    #diagonal
    res_1 = num_qubits
    #off-diagonal (using symmetry to only calculate half the sum)
    for j in range(num_qubits):
        for k in range(j + 1, num_qubits):
            res_1 += 2 * σ_2(j,k,state) * s(j) * s(k)
    res_2 = 0
    for j in  range(num_qubits):
        res_2 += σ(j,state) * s(j)
    return (res_1 - res_2**2)/num_qubits

# 4. GUI
graphical user interface to run simulations. Requires all other cells to be run first. See README for usage

In [ ]:
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from tkinter import *
from tkinter import ttk

import matplotlib
matplotlib.use("TkAgg")

'''Visualization of the results using matplotlib'''

def plot_graph_n(res_list, fun, yname):
    #time = []
    #data = []
    colormap = plt.get_cmap('plasma')
    plt.rcParams['axes.prop_cycle'] = plt.cycler(color=[colormap(k) for k in np.linspace(0, .75, len(res_list))]) 
    fig,ax = plt.subplots()
    for res in res_list:
        times_res = []
        data_res = []
        for result in res['result']:
            data_res.append(fun(result['data']))
            times_res.append(result['t'])
        ax.scatter(times_res,data_res, **{'marker': 's'})
        ax.plot(times_res,data_res, label=res['name'])
    plt.axhline(0, color='dimgray',linestyle='dashed')
    #plt.title('test') 
    ax.set_xlabel("t")
    ax.set_ylabel(yname)
    ax.legend()
    #ax.draw() 
    plt.show()
    
    
def plot_heatmap(results,fun):
    for res in results:
        data = []
        num_qubits = res['num_qubits']
        time = res['result'][len(res['result'])-1]['t']
        for result in res['result']:
            data_row = []
            for i in range(num_qubits):
                data_row.append(fun(i,result['data']))
            data.append(data_row)
        
        fig, ax = plt.subplots()
        ax.invert_yaxis()

        data = np.array(data)
        colormap = 'RdYlBu_r' 
        #colormap = 'coolwarm'
        #im = ax.imshow(data)
        hm = ax.imshow(data, origin='lower', extent=(0, num_qubits, 0, time),  cmap=colormap)
        ax.set_aspect(1.0/ax.get_data_ratio()*(data.shape[0]/num_qubits))
        #hm=ax.imshow(data/np.max(data), origin='lower', extent=(0, num_qubits, 0, time), cmap=colormap, aspect=(2*N/time))
        fig.colorbar(hm, ticks=np.arange(-1.0, 1.5, 0.5))

        ax.set_xticks(np.arange(0.5, num_qubits + 0.5, 1))
        ax.set_xticklabels(np.arange(1, num_qubits + 1, 1))

        ax.set_yticks([0,time])

        ax.set_xlabel("sites")
        ax.set_ylabel('time')

        ax.set_title(res['name'])

'''GUI using tkinter'''

# environment variables:
# num_qubits: number of qubits
# J,U,h: parameters for hamiltonian (h is a list!)
# time: total simulation time
# data_points: number of data points in a simulation
# initial_state: doamin wall (True) or neel (False)
# symmetric: do a symmetric or basic trotter decomposition?
# simulator: quantum simulator
# observable: observable for which the expectation value is calculated and plotted
# display_method: jhow to display the data (graph or heatmap)
# result: (cleaned) result of the last simulation 

results = []

#for simulation on real hardware
ibm_account_activated = False
provider = None


def run(*args):
    global results
    data = []
    run_button.configure(text='running...', command=None)
    run_button['state'] = 'disable'
    error_message.grid_remove()
    success_message.grid_remove()
    root.update()
    clean = True
    try:
        _num_qubits = int(num_qubits.get())
        _J = float(J.get())
        _U = float(U.get())
        _h_slope = float(h_slope.get())
        _h_start = float(h.get())
        _h = [_h_start + i*_h_slope for i in range(
            _num_qubits)] if _h_slope != 0. else [uniform(-_h_start, _h_start) for i in range(_num_qubits)]
        _initial_state = (neel_initial_state(_num_qubits) if initial_state.get() == 'neel' else domainwall_initial_state(_num_qubits))
        _time = float(time.get())
        _data_points = int(data_points.get())
        _symmetric = symmetric.get()
        error_message.grid_remove()
    except ValueError:
        error_message.grid(column=2, row=10, columnspan=2, sticky=(W,E))
        run_button.configure(text='RUN', command=run)
        run_button['state'] = 'normal'
        return
    try:
        match simulator.get():
            case 'numerical exact': 
                data = numerical(_num_qubits, _time, _data_points, _J, _U, _h, initial_state.get())
                clean = False
            case 'numerical trotter':
                data = numerical_trotter(_num_qubits,_symmetric, _time, _data_points, _J, _U, _h, initial_state.get())
                clean = False
            case 'numerical reduced trotter':
                data = numerical_reduced_trotter(
                    _num_qubits,_symmetric, _time, _data_points, _J, _U, _h, initial_state.get())
                clean = False
            case 'statevector simulator': data = simulate_ideal_trotter(_num_qubits, _symmetric, _time, _data_points, _J, _U, _h, _initial_state)
            case 'error free': data = simulate_ideal(_num_qubits, _symmetric, _time, _data_points,4, _J, _U, _h, _initial_state)
            case _:
                try:
                    data = simulate_real(_num_qubits, _symmetric, _time, _data_points, 4, _J, _U, _h, _initial_state,simulator.get())
                except qiskit.providers.exceptions.QiskitBackendNotFoundError:
                    data = run_on_hardware(
                        _num_qubits, _symmetric, _time, _data_points, 4, _J, _U, _h, _initial_state, simulator.get(),provider)
        last_simulation_num_qubits = num_qubits
    except qiskit.transpiler.exceptions.TranspilerError:
        error_message.grid(column=2, row=10, columnspan=2, sticky=(W, E))
        run_button.configure(text='RUN', command=run)
        run_button['state'] = 'normal'
        return
    run_button.configure(text='RUN', command=run)
    run_button['state'] = 'normal'
    success_message.grid(column=2, row=10, columnspan=2, sticky=(W,E))
    results.append({'result':clean_data_error_mitigation(data) if (error_mitigation.get() and clean) else clean_data(data) if clean else data, 'name': name.get(), 'num_qubits':_num_qubits})
    update_simulation_names()
    name.set("")

def graph(*args):
    selected_results = list(filter(lambda result: result['name'] in [
                            selected_simulations_entry.get(i) for i in selected_simulations_entry.curselection()], results))
    max_qubits = min(map(lambda x : x['num_qubits'], selected_results))
    match observable.get():
        case 'Z-Spin': 
            match display_method.get():
                case 'graph': plot_graph_n(selected_results, partial(σ,min(max_qubits - 1, int(qubit_1.get()) - 1)), 'Z-Spin')
                case _: plot_heatmap(selected_results, σ)
        case 'Domain Wall':
                plot_graph_n(selected_results, expectation_value_Nhalf,'domain wall disturbance')
        case 'Equal-time Correlator':
            match display_method.get():
                case 'graph': plot_graph_n(selected_results, partial(expectation_value_C, min(max_qubits - 1, int(qubit_1.get()) - 1), min(max_qubits - 1, int(qubit_2.get()) - 1)), 'Equal-time Correlator')
                case _: plot_heatmap(selected_results, partial(expectation_value_C, min(max_qubits - 1, int(qubit_1.get()) - 1)))
        case _:
            plot_graph_n(selected_results, expectation_value_Qf, 'QFI')


def activate_account(*args):
    global ibm_account_activated
    global provider
    try:
        try:
            IBMQ.disable_account()
        except:
            pass
        IBMQ.enable_account(ibm_token.get())
        ibm_account_activated = True
        provider = IBMQ.get_provider(hub='ibm-q')
        error_message_api.grid_remove() 
        update_available_simulators() 
        success_message_api.grid(column=4, row=0,sticky=W)

    except:
        error_message_api.grid(column=4, row=0,sticky=W)
    print('Done :D')

#window
root = Tk()
root.title("Quantum Simulator")

upframe = ttk.Frame(root, padding="3 3 12 12")
upframe.grid(column=0, row=0, sticky=(N, W, E, S))

mainframe = ttk.Frame(root, padding="3 3 12 12")
mainframe.grid(column=0, row=1, sticky=(N, W, E, S))
#ttk.Style().theme_use('clam')

style = ttk.Style(root)
root.tk.call('lappend', 'auto_path','Themes/awthemes-10.4.0')
root.tk.call('package', 'require', 'awdark')
style.theme_use('awdark')


#SECTION LOAD ACCOUNT
#IBM token
ttk.Label(upframe,text="IBM token").grid(column=0, row=0, sticky=W)
ibm_token = StringVar()
ibm_token_entry = ttk.Entry(upframe, width=34, textvariable=ibm_token)
ibm_token_entry.grid(column=1, row=0, sticky=W, pady=5, padx=5)
run_button = ttk.Button(upframe, text="activate", command=activate_account)
run_button.grid(column=2, row=0, sticky=W)
error_message_api = ttk.Label(
    upframe, text='invalid ibmq api token', foreground='#f00')
error_message_api.grid(column=3, row=0, sticky=W)
success_message_api = ttk.Label(
    upframe, text='success', foreground='#0f0')
success_message_api.grid(column=3, row=0, sticky=W)


#SECTION PHYSICS
ttk.Label(mainframe, text="PHYSICAL PROPERTIES").grid(
    column=1, row=1, sticky=W, padx=5, pady=5)

#num qubits
num_qubits = StringVar(value='6')
ttk.Label(mainframe, text="number of qubits").grid(
    column=1, row=2, sticky=W, padx=5, pady=5)
num_qubits_entry = ttk.Entry(mainframe, width=3, textvariable=num_qubits)
num_qubits_entry.grid(column=2, row=2, sticky=W, padx=5, pady=5)

#J
J = StringVar(value='1')
ttk.Label(mainframe, text="J").grid(column=1, row=3, sticky=W, padx=5, pady=5)
J_entry = ttk.Entry(mainframe, width=3, textvariable=J)
J_entry.grid(column=2, row=3, sticky=W, padx=5, pady=5)

#U
U = StringVar(value='0')
ttk.Label(mainframe, text="U").grid(column=1, row=4, sticky=W, padx=5, pady=5)
U_entry = ttk.Entry(mainframe, width=3, textvariable=U)
U_entry.grid(column=2, row=4, sticky=W, padx=5, pady=5)

#h
h = StringVar(value='0')
ttk.Label(mainframe, text="h").grid(column=1, row=5, sticky=W, padx=5, pady=5)
h_entry = ttk.Entry(mainframe, width=3, textvariable=h)
h_entry.grid(column=1, row=5, sticky=E, padx=5, pady=5)

h_slope = StringVar(value='0')
ttk.Label(mainframe,text='slope').grid(column=2, row=5, sticky=W, padx=5, pady=5)
h_slope_entry = ttk.Entry(mainframe,width=3,textvariable=h_slope)
h_slope_entry.grid(column=2, row=5, sticky=E, padx=5, pady=5)

#initial state
initial_state = StringVar(value='domain wall')
ttk.Label(mainframe, text="initial state").grid(
    column=1, row=6, sticky=W, padx=5, pady=5)
initial_state_entry = ttk.Combobox(
    mainframe, textvariable=initial_state,  state='readonly')
initial_state_entry.grid(column=2, row=6, sticky=W,padx=5, pady=5)
initial_state_entry['values'] = ['domain wall','neel']

#SECTION SIMULATION
ttk.Label(mainframe, text="SIMULATION").grid(
    column=3, row=1, sticky=W, padx=5, pady=5)

#time
time = StringVar(value='2.5')
ttk.Label(mainframe, text="time").grid(
    column=3, row=2, sticky=W, padx=5, pady=5)
time_entry = ttk.Entry(mainframe, width=3, textvariable=time)
time_entry.grid(column=4, row=2, sticky=W, padx=5, pady=5)

#data points
data_points = StringVar(value='16')
ttk.Label(mainframe, text="data points").grid(
    column=3, row=3, sticky=W, padx=5, pady=5)
data_points_entry = ttk.Entry(mainframe, width=3, textvariable=data_points)
data_points_entry.grid(column=4, row=3, sticky=W, padx=5, pady=5)

#symmetric
symmetric = BooleanVar()
symmetric_label = ttk.Label(mainframe, text="symmetric trotter decomposition")
symmetric_label.grid(column=3, row=4, sticky=W, padx=5, pady=5)
symmetric_entry = ttk.Checkbutton(mainframe, variable=symmetric)
symmetric_entry.grid(column=4, row=4, sticky=W, padx=5, pady=5)

#error mitigation
error_mitigation = BooleanVar()
error_mitigation_label = ttk.Label(mainframe, text="error mitigation")
error_mitigation_label.grid(column=3, row=5, sticky=W, padx=5, pady=5)
error_mitigation_entry = ttk.Checkbutton(mainframe,variable=error_mitigation)
error_mitigation_entry.grid(column=4, row=5, sticky=W, padx=5, pady=5)

#simulator
simulator = StringVar(value='error free')
ttk.Label(mainframe, text="simulator").grid(column=3, row=6, sticky=W, padx=5, pady=5)

#name
name=StringVar(value='new Simulation')
ttk.Label(mainframe, text="name").grid(column=2, row=7, sticky=E, pady=(20,0), padx=5)
name_entry = ttk.Entry(mainframe, textvariable=name)
name_entry.grid(column=3, row=7, sticky=W,pady=(20,0), padx=5)

#show only quantum computers with enpough qubits 
def update_available_simulators():
    _list = ['numerical exact', 'numerical trotter',
             'numerical reduced trotter', 'statevector simulator', 'error free']
    if ibm_account_activated:
            _list.extend([p.name() for p in provider.backends(simulator=False, operational=True)])
    try:
        N = int(num_qubits.get())
        for fake_device in qiskit.test.mock.FakeProvider().backends():
            if fake_device.configuration().n_qubits >= N:
                _list.append(fake_device.name())
    except ValueError:
        print("value error :(")
        pass
    simulator_entry["values"] = _list

#disable trotter ad error correction options if numerical simulation is selected
def update_simulation_options(*args):
    if(simulator.get() == 'numerical exact'):
        symmetric_entry['state'] = 'disable'
        error_mitigation_entry['state'] = 'disable'
        error_mitigation_label.config(foreground='#888')
        symmetric_label.config(foreground='#888')

    else:
        symmetric_entry['state'] = 'normal'
        error_mitigation_entry['state'] = 'normal'
        error_mitigation_label.config(foreground='#fff')
        symmetric_label.config(foreground='#fff')


simulator_entry = ttk.Combobox(mainframe, textvariable=simulator, state='readonly', postcommand=update_available_simulators)
simulator_entry.grid(column=4, row=6, sticky=W, padx=5, pady=5)
simulator_entry['values'] = ['numerical exact','numerical trotter', 'numerical reduced trotter','statevector simulator', 'error free']
simulator_entry.bind("<<ComboboxSelected>>", update_simulation_options)


#RUN BUTTON
run_button = ttk.Button(mainframe, text="RUN", command=run)
run_button.grid(column=2, row=8, columnspan=2, sticky=(W, E))
run_button.grid_configure(padx=100, pady=(20, 5))
error_message = ttk.Label(mainframe, text="Invalid input", foreground='#f00', anchor='center')
error_message.grid(column=2, row=9, columnspan=2,
                   sticky=(W, E), padx=5, pady=5)

success_message = ttk.Label(mainframe, text="Done",foreground='#0f0', anchor='center')
success_message.grid(column=2, row=9, columnspan=2,
                     sticky=(W, E), padx=5, pady=5)


#SECTION DATA
ttk.Label(mainframe, text="DATA PROCESSING").grid(column=1, row=11, sticky=W, padx=5, pady=5)


def manage_display_options(event):
    obs = observable.get()
    method = display_method.get()
    if ((obs == 'QFI') or (obs == 'Domain Wall')):
        display_method_entry['values'] = ['graph']
        display_method_entry.set('graph')
    else:
        display_method_entry['values'] = ['graph', 'heatmap']
    match obs, method:
        case(('QFI', _) | ('Domain Wall', _) | ('Z-Spin', 'heatmap')):
            qubit_1_entry['state'] = 'disabled'
            qubit_2_entry['state'] = 'disabled'
            qubit_1_label.config(foreground='#888')
            qubit_2_label.config(foreground='#888')
        case(('Z-Spin', 'graph') | ('Equal-time Correlator', 'heatmap')):
            qubit_1_entry['state'] = 'normal'
            qubit_2_entry['state'] = 'disabled'
            qubit_1_label.config(foreground='#fff')
            qubit_2_label.config(foreground='#888')
        case('Equal-time Correlator', 'graph'):
            qubit_1_entry['state'] = 'normal'
            qubit_2_entry['state'] = 'normal'
            qubit_1_label.config(foreground='#fff')
            qubit_2_label.config(foreground='#fff')

#expectation value
observable = StringVar(value='Z-Spin')
ttk.Label(mainframe, text="observable").grid(
    column=1, row=12, sticky=W, padx=5, pady=5)
observable_entry = ttk.Combobox(
    mainframe, textvariable=observable,  state='readonly')
observable_entry.grid(column=2, row=12, sticky=W, padx=5, pady=5)
observable_entry['values'] = ['Z-Spin', 'Domain Wall', 'Equal-time Correlator', 'QFI']
observable_entry.bind("<<ComboboxSelected>>", manage_display_options)

#select qubits (indexing starts at 1)
qubit_1_label = ttk.Label(mainframe, text="qubit 1")
qubit_1_label.grid(column=1, row=13, sticky=W, padx=5, pady=5)
qubit_2_label = ttk.Label(mainframe, text="qubit 2", foreground='#888')
qubit_2_label.grid(column=2, row=13, sticky=W, padx=5, pady=5)

#selectable qubits only go up to the minimum of all selected simulations
def update_qubit_options(*args):
    max_qubit = min(map(lambda result : result['num_qubits'],filter(lambda result: result['name'] in [
                            selected_simulations_entry.get(i) for i in selected_simulations_entry.curselection()], results)), default=0)
    try:
        values = [i for i in range(1,max_qubit + 1)]
        qubit_1_entry['values'] = values
        qubit_2_entry['values'] = values
    except ValueError:
        pass

qubit_1 = StringVar(value='1')
qubit_1_entry = ttk.Combobox(mainframe, textvariable=qubit_1,state='readonly', postcommand=update_qubit_options, width=3)
qubit_1_entry.grid(column=1, row=14, sticky=W, padx=5, pady=5)
qubit_2 = StringVar(value='1')
qubit_2_entry = ttk.Combobox(mainframe, textvariable=qubit_2,
                             state='readonly', postcommand=update_qubit_options, width=3)
qubit_2_entry.grid(column=2, row=14, sticky=W, padx=5, pady=5)
qubit_2_entry['state'] = 'disabled'



#display method
display_method = StringVar(value='graph')
ttk.Label(mainframe, text="display method").grid(column=1, row=15, sticky=W, padx=5, pady=5)
display_method_entry = ttk.Combobox(
    mainframe, textvariable=display_method,  state='readonly')
display_method_entry.grid(column=2, row=15, sticky=W, padx=5, pady=5)
display_method_entry['values'] = ['graph', 'heatmap']
display_method_entry.bind("<<ComboboxSelected>>", manage_display_options)
display_method_entry.bind("<<ComboboxSelected>>", update_qubit_options, add='+')

#simulation selection
selected_simulations = StringVar()
ttk.Label(mainframe, text="select results to plot").grid(
    column=4, row=11, sticky=W, padx=5, pady=5)
selected_simulations_entry = Listbox(
    mainframe, listvariable=selected_simulations, selectmode='extended')
selected_simulations_entry.grid(column=4, row=12,columnspan=4, rowspan=4, sticky=W, padx=5, pady=5)
selected_simulations_entry.bind('<<ListboxSelect>>', update_qubit_options)

def update_simulation_names():
    selected_simulations.set(list(map(lambda res: res['name'], results)))


#plot BUTTON
plot_button = ttk.Button(mainframe, text="SHOW", command=graph)
plot_button.grid(column=2, row=16,columnspan=2, sticky=(W, E))
plot_button.grid_configure(padx=100, pady=(20, 5))


error_message.grid_remove()
success_message.grid_remove()
error_message_api.grid_remove()
success_message_api.grid_remove()

root.mainloop()


# Sources
[1] Smith, A., Kim, M.S., Pollmann, F. et al. Simulating quantum many-body dynamics on a current digital quantum computer. npj Quantum Inf 5, 106 (2019). https://doi.org/10.1038/s41534-019-0217-0

[2] [IBM Quantum](https://quantum-computing.ibm.com/), 2021

[3] tkinter theme: [awthemes](https://sourceforge.net/projects/tcl-awthemes/)